# xml 안에서 데이터 추출
- 원하는 데이터들만 json type 으로 변경하고 그에 대한 분포를 dataframe 형태로 만들자
- 필요없는 이미지 리스트를 만들어 놓는게 좋을까 고민은 해볼 것(data root 경로를 뺀 상태에서 이미지 리스트를 파일로 저장해두자)

In [ ]:
XML_DIR = '/opt/ml/data/team_github/xml'
XML_LIST = [i for i in os.listdir(XML_DIR) if i.endswith('.xml')]
print(len(XML_LIST))
print(XML_LIST[0])

In [ ]:
all_info = []
from tqdm import tqdm
for x in tqdm(XML_LIST):
    all_info.extend(extract_xml_info(x))
print()
print(len(all_info))

In [ ]:
# 전체 정보를 가져와서 data frame 형태로 만들어야 한다. -> 그래야 추후에 이미지 검증에도 활용 가능하다
# 그 다음에 필요한 클래스를 가지는 좌표를 가진 이미지만 쿼리를 통해서 빼낸다.
# 주석 처리한 print 는 검증용이니 확인해보셔도 좋을 것 같아요
def extract_xml_info(xml_file):
    with open(os.path.join(XML_DIR, xml_file), 'r') as f:
        #xml 모양의 str 이어야 동작된다.
        raw_xml = xmltodict.parse(f.read())
        #자주 사용한 json dict 로 변환하기 위함(ordereddict 가 반환되어 그렇슴)
        raw_xml = json.dumps(raw_xml)
        xml_to_json = json.loads(raw_xml)

    root_folder, sub_folder, xml_name = xml_file.split('-')
    # print(root_folder, sub_folder, xml_name)
    # print(f"#images : {len(xml_to_json['annotations']['image'])}")
    #for dataframe
    return_list = []
    # 이미지 별로
    # box_count = 0

    for image in xml_to_json['annotations']['image']:
        # 박스 별로 @occluded @label [@xtl,@ytl,@xbr,@ybr]
        temp_list = []
        
        if type(image['box']) != list:
            # extend 하는 경우에 리스트가 풀어져서 합쳐지기 때문에 dim=2 여야한다.
            temp_list = [[root_folder, sub_folder, xml_name, image['@name'],
                        image['box']['@occluded'], image['box']['@label'],
                        [image['box']['@xtl'], image['box']['@ytl'], image['box']['@xbr'], image['box']['@ybr']]
                        ]]
        else:
            for box in image['box']:
                box_info = [root_folder, sub_folder, xml_name, image['@name'],,
                box['@occluded'], box['@label'],,
                [box['@xtl'], box['@ytl'], box['@xbr'], box['@ybr']],
                ]
            temp_list.append(box_info)

        # 모은 정보를 리턴할 리스트에 extend
        # box_count += len(temp_list)
        return_list.extend(temp_list)
    # print(f'before extracting #box : {box_count}\t after #box : {len(return_list)}')
    return return_list

In [ ]:
import pandas as pd

columns = ['root_folder', 'sub_folder', 'xml_name', 'file_name', 'occluded', 'classes', 'box']
target_list = ('car', 'person', 'traffic_light', 'truck', 'bicycle', 'motorcycle', 'bus', 'carrier', 'stroller', 'scooter', 'wheelchair')

pandas_dict = pd.DataFrame(all_info, columns=columns)
target_frame = pandas_dict[pandas_dict['classes'].isin(target_list)]
print(f'total data : {len(pandas_dict)}\\t target data : {len(target_frame)}')

pandas_dict.to_csv('crosswalk_temp.csv' ,header=True, index=False)

In [ ]:
target_frame.to_csv('XML_filtered(21_30).csv', header=True, index=False)